In [ ]:
import random
import numpy as np
import pandas as pd
import re
import json
from pathlib import Path

import os
import shutil

import matplotlib.pyplot as plt
import torch


import torchvision

from torch import nn
from torchvision import transforms, datasets
# import torchvision.transforms.functional as F
from torch.utils.data import DataLoader
from torchinfo import summary

from PIL import Image

# split train and test

In [ ]:
# import os
# import glob
# import random
# import re
# import shutil

# # Set seed for reproducibility
# random.seed(42)

# # Define base directories
# base_dir = "/Users/bin/Desktop/CV_Assignment/Dataset_filtered"
# trainval_dir = os.path.join(base_dir, "TrainVal")
# color_dir = os.path.join(trainval_dir, "color")
# label_dir = os.path.join(trainval_dir, "label")

# # Define destination directories for train and validation splits
# train_color_dir = os.path.join(base_dir, "train", "color")
# train_label_dir = os.path.join(base_dir, "train", "label")
# val_color_dir = os.path.join(base_dir, "val", "color")
# val_label_dir = os.path.join(base_dir, "val", "label")

# # Create the destination directories if they don't exist
# for d in [train_color_dir, train_label_dir, val_color_dir, val_label_dir]:
#     os.makedirs(d, exist_ok=True)

# # Find all jpg files in the color folder
# image_files = glob.glob(os.path.join(color_dir, "*.jpg"))

# # Group files by animal name (assumes format like "Abyssinian_1.jpg")
# groups = {}
# pattern = re.compile(r'(.+)_\d+\.jpg')  # capture group for the animal name
# for image_file in image_files:
#     basename = os.path.basename(image_file)
#     match = pattern.match(basename)
#     if match:
#         animal = match.group(1)
#     else:
#         animal = "unknown"
#     groups.setdefault(animal, []).append(basename)

# # Split the files for each animal into train (80%) and val (20%)
# train_list = []
# val_list = []
# for animal, files in groups.items():
#     random.shuffle(files)
#     split_index = int(0.8 * len(files))
#     train_files = files[:split_index]
#     val_files = files[split_index:]
#     train_list.extend(train_files)
#     val_list.extend(val_files)

# # Define output text file paths
# train_txt_path = os.path.join(base_dir, "train.txt")
# val_txt_path = os.path.join(base_dir, "val.txt")

# # Copy files to new folders and write paths to train.txt and val.txt
# with open(train_txt_path, "w") as train_file, open(val_txt_path, "w") as val_file:
#     # Process train split
#     for filename in train_list:
#         # Copy color image
#         src_color = os.path.join(color_dir, filename)
#         dst_color = os.path.join(train_color_dir, filename)
#         shutil.copy2(src_color, dst_color)
        
#         # Derive corresponding label filename (change extension from .jpg to .png)
#         label_filename = filename.replace(".jpg", ".png")
#         src_label = os.path.join(label_dir, label_filename)
#         dst_label = os.path.join(train_label_dir, label_filename)
#         if os.path.exists(src_label):
#             shutil.copy2(src_label, dst_label)
        
#         # Write relative paths to train.txt (format: "train/color/<filename> train/label/<label_filename>")
#         train_file.write(f"{os.path.join('train','color',filename)} {os.path.join('train','label',label_filename)}\n")
    
#     # Process validation split
#     for filename in val_list:
#         # Copy color image
#         src_color = os.path.join(color_dir, filename)
#         dst_color = os.path.join(val_color_dir, filename)
#         shutil.copy2(src_color, dst_color)
        
#         # Corresponding label filename
#         label_filename = filename.replace(".jpg", ".png")
#         src_label = os.path.join(label_dir, label_filename)
#         dst_label = os.path.join(val_label_dir, label_filename)
#         if os.path.exists(src_label):
#             shutil.copy2(src_label, dst_label)
        
#         # Write relative paths to val.txt
#         val_file.write(f"{os.path.join('val','color',filename)} {os.path.join('val','label',label_filename)}\n")

# print("Data splitting and copying complete.")
# print(f"Train list written to {train_txt_path}")
# print(f"Val list written to {val_txt_path}")


# get statistics of the split

In [ ]:
# import os
# import re
# from collections import Counter

# def extract_animal(filepath):
#     """
#     Extracts the animal class from a filename.
#     Assumes filename format: AnimalName_123.jpg
#     """
#     # Use regex to extract animal class from the basename.
#     pattern = re.compile(r'(.+)_\d+\.jpg')
#     basename = os.path.basename(filepath)
#     match = pattern.match(basename)
#     if match:
#         return match.group(1)
#     else:
#         return "unknown"

# def get_statistics(txt_file):
#     """
#     Reads the text file (each line with two paths separated by whitespace)
#     and returns the total count and a Counter with the distribution of animal classes.
#     """
#     counter = Counter()
#     total = 0
#     with open(txt_file, "r") as f:
#         for line in f:
#             line = line.strip()
#             if line:
#                 parts = line.split()
#                 if len(parts) != 2:
#                     print(f"Warning: Unexpected line format: {line}")
#                     continue
#                 # Extract the animal from the color image path (first column)
#                 animal = extract_animal(parts[0])
#                 counter[animal] += 1
#                 total += 1
#     return total, counter

# def print_statistics(txt_file):
#     total, counter = get_statistics(txt_file)
#     print(f"Statistics for {txt_file}:")
#     print(f"  Total samples: {total}")
#     print("  Distribution by animal class:")
#     for animal, count in counter.items():
#         print(f"    {animal}: {count}")

# if __name__ == "__main__":
#     # Update base_dir if needed
#     base_dir = "/Users/bin/Desktop/CV_Assignment/Dataset_filtered"
#     train_txt_path = os.path.join(base_dir, "train.txt")
#     val_txt_path = os.path.join(base_dir, "val.txt")
    
#     print_statistics(train_txt_path)
#     print()
#     print_statistics(val_txt_path)


# check image dimension statistics

In [ ]:
# import os
# import cv2
# import glob
# from collections import Counter

# # Specify the directory containing your images.
# # You can change this to any folder you'd like to analyze.
# image_folder = "/Users/bin/Desktop/CV_Assignment/Dataset_filtered/TrainVal/color"

# # Find all jpg files in the folder
# image_files = glob.glob(os.path.join(image_folder, "*.jpg"))

# resolutions = []  # List to store (width, height) tuples

# for img_path in image_files:
#     img = cv2.imread(img_path)
#     if img is None:
#         print("Could not read:", img_path)
#         continue
#     height, width = img.shape[:2]
#     resolutions.append((width, height))

# # If any images were found, calculate statistics.
# if resolutions:
#     widths = [w for w, h in resolutions]
#     heights = [h for w, h in resolutions]
#     avg_width = sum(widths) / len(widths)
#     avg_height = sum(heights) / len(heights)
#     min_width = min(widths)
#     max_width = max(widths)
#     min_height = min(heights)
#     max_height = max(heights)
    
#     # Count unique resolution occurrences
#     resolution_counter = Counter(resolutions)
    
#     print("Total images:", len(resolutions))
#     print(f"Average resolution: {int(avg_width)} x {int(avg_height)}")
#     print(f"Minimum width: {min_width}, Minimum height: {min_height}")
#     print(f"Maximum width: {max_width}, Maximum height: {max_height}")
#     print("\nUnique resolutions (width x height) and counts:")
#     for res, count in resolution_counter.items():
#         print(f"{res[0]} x {res[1]}: {count}")
# else:
#     print("No images found in the specified folder.")


# Inspect image


In [ ]:
# import cv2
# import matplotlib.pyplot as plt

# # Path to an example image
# image_path = "/Users/bin/Desktop/CV_Assignment/Dataset_filtered/TrainVal/color/yorkshire_terrier_144.jpg"

# # Load the original image (OpenCV loads in BGR format)
# original = cv2.imread(image_path, cv2.IMREAD_COLOR)
# if original is None:
#     raise ValueError("Could not load the image. Please check the image path.")

# # Convert the image from BGR to RGB for display purposes
# original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)

# # Define the target size (for example, 256x256)
# TARGET_WIDTH, TARGET_HEIGHT = 256, 256
# target_size = (TARGET_WIDTH, TARGET_HEIGHT)

# # Resize the image using bilinear interpolation
# resized = cv2.resize(original_rgb, target_size, interpolation=cv2.INTER_LINEAR)

# # Display the original and resized images side by side
# plt.figure(figsize=(12, 6))

# plt.subplot(1, 2, 1)
# plt.imshow(original_rgb)
# plt.title("Original Image")
# plt.axis("off")

# plt.subplot(1, 2, 2)
# plt.imshow(resized)
# plt.title("Resized (Interpolated) Image")
# plt.axis("off")

# plt.show()


# Resize images

initialize一个dictionary，对每个image resize之前，记录它的height和width并写入字典，最后将字典写入一个叫original_sizes.json的file里

In [6]:
import os
import cv2
import shutil
import json

# Set target dimensions (adjust as needed)
TARGET_WIDTH = 256
TARGET_HEIGHT = 256
target_size = (TARGET_WIDTH, TARGET_HEIGHT)

# Base directory (adjust this to your setup)
base_dir = "/Users/bin/Desktop/CV_Assignment/Dataset_filtered"

# Directories for train and val splits
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")

# Create destination directories:
# For training, we resize images but copy masks unchanged.
resized_train_color_dir = os.path.join(base_dir, "train_resized", "color")
resized_train_label_dir = os.path.join(base_dir, "train_resized", "label")

# For validation, we resize images and copy masks unchanged.
resized_val_color_dir = os.path.join(base_dir, "val_resized", "color")
resized_val_label_dir = os.path.join(base_dir, "val_resized", "label")

for d in [resized_train_color_dir, resized_train_label_dir,
          resized_val_color_dir, resized_val_label_dir]:
    os.makedirs(d, exist_ok=True)

# Dictionary to store original image sizes before resizing
original_sizes = {}

def resize_and_save(src_path, dst_path, target_size, original_sizes_dict):
    """
    Resize an image, save it, and record its original size.
    """
    img = cv2.imread(src_path, cv2.IMREAD_COLOR)
    if img is None:
        print("Error reading:", src_path)
        return
    
    # Store original size before resizing
    img_name = os.path.basename(src_path).replace(".jpg", "")  # Remove file extension
    original_sizes_dict[img_name] = img.shape[:2]  # Store (height, width)

    # Resize and save
    resized = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
    cv2.imwrite(dst_path, resized)

def process_data(color_source, label_source, resized_color_dest, resized_label_dest, original_sizes_dict):
    """
    Processes a dataset split:
    - Resizes color images and stores original sizes.
    - Copies label masks unchanged.
    """
    # Resize color images
    for filename in sorted(os.listdir(color_source)):
        if filename.lower().endswith(".jpg"):
            src_path = os.path.join(color_source, filename)
            dst_path = os.path.join(resized_color_dest, filename)
            resize_and_save(src_path, dst_path, target_size, original_sizes_dict)

    # Copy masks unchanged
    for filename in sorted(os.listdir(label_source)):
        if filename.lower().endswith(".png"):
            src_path = os.path.join(label_source, filename)
            dst_path = os.path.join(resized_label_dest, filename)
            shutil.copy2(src_path, dst_path)

# -----------------------------------------
# Process Training Data
# -----------------------------------------
train_color_source = os.path.join(train_dir, "color")
train_label_source = os.path.join(train_dir, "label")

process_data(train_color_source, train_label_source, resized_train_color_dir, resized_train_label_dir, original_sizes)

# -----------------------------------------
# Process Validation Data
# -----------------------------------------
val_color_source = os.path.join(val_dir, "color")
val_label_source = os.path.join(val_dir, "label")

process_data(val_color_source, val_label_source, resized_val_color_dir, resized_val_label_dir, original_sizes)

# Save original sizes to a JSON file
original_size_json_path = os.path.join(base_dir, "original_sizes.json")
with open(original_size_json_path, "w") as f:
    json.dump(original_sizes, f, indent=4)

print(f"Processing complete. Original sizes saved to {original_size_json_path}.")

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 240 extraneous bytes before marker 0xd9


Processing complete. Original sizes saved to /Users/bin/Desktop/CV_Assignment/Dataset_filtered/original_sizes.json.


# randaugment (Do the augmentations)


In [ ]:
# import os
# import random
# from PIL import Image, ImageOps, ImageEnhance
# import math
# from torchvision.transforms import functional as F
# from torchvision.transforms.functional import InterpolationMode

# # Define the RandAugment operation pool (14 ops as in the original paper)
# RA_OPERATIONS = [
#     "Identity", "AutoContrast", "Equalize",
#     "Rotate", "Solarize", "Color", "Posterize",
#     "Contrast", "Brightness", "Sharpness",
#     "ShearX", "ShearY", "TranslateX", "TranslateY"
# ]

# def apply_operation(img, mask, op_name, magnitude):
#     """
#     Apply a single augmentation operation to the image (and mask, if applicable).
#     'magnitude' is on a 0-10 scale indicating severity.
#     """
#     if op_name == "Identity":
#         return img, mask  # no change

#     if op_name == "AutoContrast":
#         img = ImageOps.autocontrast(img)
#         return img, mask

#     if op_name == "Equalize":
#         img = ImageOps.equalize(img)
#         return img, mask

#     if op_name == "Rotate":
#         max_deg = 30.0
#         angle = magnitude / 10.0 * max_deg
#         if random.random() < 0.5:
#             angle = -angle
#         img = img.rotate(angle, resample=Image.BILINEAR, fillcolor=0)
#         mask = mask.rotate(angle, resample=Image.NEAREST, fillcolor=0)
#         return img, mask

#     if op_name == "Solarize":
#         thresh = int(256 - (magnitude / 10.0) * 256)
#         img = ImageOps.solarize(img, thresh)
#         return img, mask

#     if op_name == "Color":
#         factor = 1.0 + (magnitude / 10.0) * 0.9
#         if random.random() < 0.5:
#             factor = 1.0 - (magnitude / 10.0) * 0.9
#         img = ImageEnhance.Color(img).enhance(factor)
#         return img, mask

#     if op_name == "Posterize":
#         bits = int(round(8 - (magnitude / 10.0) * 4))
#         bits = max(1, bits)
#         img = ImageOps.posterize(img, bits)
#         return img, mask

#     if op_name == "Contrast":
#         factor = 1.0 + (magnitude / 10.0) * 0.9
#         if random.random() < 0.5:
#             factor = 1.0 - (magnitude / 10.0) * 0.9
#         img = ImageEnhance.Contrast(img).enhance(factor)
#         return img, mask

#     if op_name == "Brightness":
#         factor = 1.0 + (magnitude / 10.0) * 0.9
#         if random.random() < 0.5:
#             factor = 1.0 - (magnitude / 10.0) * 0.9
#         img = ImageEnhance.Brightness(img).enhance(factor)
#         return img, mask

#     if op_name == "Sharpness":
#         factor = 1.0 + (magnitude / 10.0) * 0.9
#         if random.random() < 0.5:
#             factor = 1.0 - (magnitude / 10.0) * 0.9
#         img = ImageEnhance.Sharpness(img).enhance(factor)
#         return img, mask

#     # Geometric operations: these need to be applied identically to the mask
#     if op_name == "ShearX":
#         shear_factor = magnitude / 10.0 * 0.3
#         if random.random() < 0.5:
#             shear_factor = -shear_factor
#         shear_degrees = math.degrees(math.atan(shear_factor))
#         img = F.affine(img, angle=0.0, translate=(0, 0), scale=1.0,
#                        shear=(shear_degrees, 0.0),
#                        interpolation=InterpolationMode.BILINEAR, fill=0)
#         mask = F.affine(mask, angle=0.0, translate=(0, 0), scale=1.0,
#                         shear=(shear_degrees, 0.0),
#                         interpolation=InterpolationMode.NEAREST, fill=0)
#         return img, mask

#     if op_name == "ShearY":
#         shear_factor = magnitude / 10.0 * 0.3
#         if random.random() < 0.5:
#             shear_factor = -shear_factor
#         shear_degrees = math.degrees(math.atan(shear_factor))
#         img = F.affine(img, angle=0.0, translate=(0, 0), scale=1.0,
#                        shear=(0.0, shear_degrees),
#                        interpolation=InterpolationMode.BILINEAR, fill=0)
#         mask = F.affine(mask, angle=0.0, translate=(0, 0), scale=1.0,
#                         shear=(0.0, shear_degrees),
#                         interpolation=InterpolationMode.NEAREST, fill=0)
#         return img, mask

#     if op_name == "TranslateX":
#         max_frac = 0.45
#         dx = int(round(magnitude / 10.0 * max_frac * img.width))
#         if random.random() < 0.5:
#             dx = -dx
#         img = F.affine(img, angle=0.0, translate=(dx, 0), scale=1.0,
#                        shear=(0.0, 0.0), interpolation=InterpolationMode.BILINEAR, fill=0)
#         mask = F.affine(mask, angle=0.0, translate=(dx, 0), scale=1.0,
#                         shear=(0.0, 0.0), interpolation=InterpolationMode.NEAREST, fill=0)
#         return img, mask

#     if op_name == "TranslateY":
#         max_frac = 0.45
#         dy = int(round(magnitude / 10.0 * max_frac * img.height))
#         if random.random() < 0.5:
#             dy = -dy
#         img = F.affine(img, angle=0.0, translate=(0, dy), scale=1.0,
#                        shear=(0.0, 0.0), interpolation=InterpolationMode.BILINEAR, fill=0)
#         mask = F.affine(mask, angle=0.0, translate=(0, dy), scale=1.0,
#                         shear=(0.0, 0.0), interpolation=InterpolationMode.NEAREST, fill=0)
#         return img, mask

#     return img, mask

# def randaugment_image_mask(img, mask, N, M):
#     """Apply RandAugment (N operations with magnitude M) to the given PIL image and mask."""
#     ops = random.sample(RA_OPERATIONS, N)  # choose N distinct ops at random
#     for op in ops:
#         img, mask = apply_operation(img, mask, op, M)
#     return img, mask

# def run_randaugment_train_resized(input_base, output_base, N=2, M=9, num_aug=3):
#     """
#     Process each image-mask pair from the resized training data.
    
#     Expects:
#       - Images in:  input_base/train_resized/color
#       - Masks in:   input_base/train_resized/label
    
#     Saves augmented outputs in:
#       - output_base/train_randaugmented/color
#       - output_base/train_randaugmented/label
    
#     Additionally, copies the original image and mask.
#     Generates 'num_aug' augmented images per original image.
#     """
#     in_img_dir = os.path.join(input_base, "train_resized", "color")
#     in_mask_dir = os.path.join(input_base, "train_resized", "label")
#     out_img_dir = os.path.join(output_base, "train_randaugmented", "color")
#     out_mask_dir = os.path.join(output_base, "train_randaugmented", "label")
#     os.makedirs(out_img_dir, exist_ok=True)
#     os.makedirs(out_mask_dir, exist_ok=True)

#     for filename in os.listdir(in_img_dir):
#         if not filename.lower().endswith(".jpg"):
#             continue
#         img_path = os.path.join(in_img_dir, filename)
#         base_name, ext = os.path.splitext(filename)
#         # Try common mask extensions (often masks are stored as .png)
#         mask_candidates = [base_name + ".png", base_name + ".jpg", base_name + ".jpeg", base_name + ".bmp", base_name + ".tif"]
#         mask_path = None
#         for cand in mask_candidates:
#             cand_path = os.path.join(in_mask_dir, cand)
#             if os.path.exists(cand_path):
#                 mask_path = cand_path
#                 break
#         if mask_path is None:
#             print(f"Mask for image {filename} not found, skipping.")
#             continue

#         img = Image.open(img_path).convert("RGB")
#         mask = Image.open(mask_path)
#         if mask.mode not in ["L", "I"]:
#             mask = mask.convert("L")

#         # Save the original image and mask in the output folders
#         orig_img_path = os.path.join(out_img_dir, f"{base_name}_orig.jpg")
#         orig_mask_path = os.path.join(out_mask_dir, f"{base_name}_orig.png")
#         img.save(orig_img_path)
#         mask.save(orig_mask_path)
#         print(f"Saved original image to {orig_img_path} and mask to {orig_mask_path}")

#         # Generate and save augmented versions
#         for i in range(num_aug):
#             aug_img, aug_mask = randaugment_image_mask(img, mask, N, M)
#             out_img_path = os.path.join(out_img_dir, f"{base_name}_aug_{i}.jpg")
#             out_mask_path = os.path.join(out_mask_dir, f"{base_name}_aug_{i}.png")
#             aug_img.save(out_img_path)
#             aug_mask.save(out_mask_path)
#             print(f"Saved augmented image to {out_img_path} and mask to {out_mask_path}")

# # --- Example usage in a notebook cell ---
# # Set your base directory. In your case, it is:
# base_dir = "/Users/bin/Desktop/CV_Assignment/Dataset_filtered"
# # We'll use the same base directory for output (augmented data will be saved under a new subfolder)
# run_randaugment_train_resized(base_dir, base_dir, N=2, M=9, num_aug=3)


将images和masks都写入DataLoader里，但同时也要记录每张image的name以便后面查询mask的original size

In [10]:
from custom_dataset import CustomDataset
# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize only for training input
    transforms.ToTensor()
])

# Create dataset
dataset = CustomDataset(
    image_dir="/Users/bin/Desktop/CV_Assignment/Dataset_filtered/train_randaugmented/color",
    mask_dir="/Users/bin/Desktop/CV_Assignment/Dataset_filtered/train_randaugmented/label",
    transform=transform
)

# Custom collate function to handle filenames
def custom_collate_fn(batch):
    images, masks, img_names, mask_names = zip(*batch)

    img_names = [re.sub(r'(_aug_\d+|_orig)$', '', name) for name in img_names]
    mask_names = [re.sub(r'(_aug_\d+|_orig)$', '', name) for name in mask_names]

    images = torch.stack(images)
    masks = torch.stack(masks)
    return images, masks, list(img_names), list(mask_names)

# Create DataLoader with shuffle=True but keeping filenames
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0, 
                              pin_memory=True, collate_fn=custom_collate_fn)

# Test DataLoader
for images, masks, img_names, mask_names in train_dataloader:
    print("Batch Image Filenames:", img_names)
    print("Batch Mask Filenames:", mask_names)
    break

Batch Image Filenames: ['Abyssinian_10', 'japanese_chin_172', 'boxer_139', 'samoyed_162', 'basset_hound_142', 'havanese_105', 'miniature_pinscher_120', 'saint_bernard_115', 'Birman_102', 'scottish_terrier_152', 'havanese_103', 'chihuahua_137', 'samoyed_14', 'Maine_Coon_176', 'Bombay_12', 'basset_hound_157', 'basset_hound_108', 'Sphynx_196', 'leonberger_123', 'english_cocker_spaniel_158', 'Russian_Blue_162', 'miniature_pinscher_122', 'newfoundland_128', 'Siamese_147', 'english_cocker_spaniel_117', 'Russian_Blue_116', 'basset_hound_112', 'pomeranian_148', 'great_pyrenees_188', 'saint_bernard_155', 'saint_bernard_182', 'saint_bernard_157']
Batch Mask Filenames: ['Abyssinian_10', 'japanese_chin_172', 'boxer_139', 'samoyed_162', 'basset_hound_142', 'havanese_105', 'miniature_pinscher_120', 'saint_bernard_115', 'Birman_102', 'scottish_terrier_152', 'havanese_103', 'chihuahua_137', 'samoyed_14', 'Maine_Coon_176', 'Bombay_12', 'basset_hound_157', 'basset_hound_108', 'Sphynx_196', 'leonberger_1

In [ ]:
from unet import UNet
import torch.optim as optim

model = UNet(in_channels=3, out_channels=1)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# summary(model=model, 
#         input_size=(32, 3, 256, 256), # make sure this is "input_size", not "input_shape"
#         # col_names=["input_size"], # uncomment for smaller output
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
# ) 

x = torch.randn(32, 3, 256, 256)  # Example input tensor
output = model(x)
print(output.shape)  # Expected: torch.Size([1, 1, 256, 256])

In [ ]:
def resize_logits_before_argmax(logits, original_sizes):
    """
    Resize logits to their original image sizes before applying argmax.
    
    Args:
        logits (torch.Tensor): Predicted logits of shape (batch_size, 1, 256, 256).
        original_sizes (list of tuples): List of (H, W) for each image in the batch.

    Returns:
        list: A list of resized masks [(1, H1, W1), (1, H2, W2), ...], each with its corresponding original size.
    """
    resized_masks = []
    for i in range(len(logits)):  
        orig_h, orig_w = original_sizes[i]  # Get the original height and width for each mask
        resized_logit = F.interpolate(logits[i].unsqueeze(0),  # (1, 256, 256) → (1, H, W)
                                      size=(orig_h, orig_w), 
                                      mode="bilinear", 
                                      align_corners=False)  # Bilinear interpolation ensures smooth resizing

        # **Apply thresholding to convert logits into binary masks (0/1)**
        binary_mask = (resized_logit > 0.5).float()  # Thresholding to obtain binary mask
        resized_masks.append(binary_mask)  # Store the resized binary mask in a list

    return resized_masks  # Return a list containing masks of different sizes


# Define the loss function
loss_fn = torch.nn.BCEWithLogitsLoss()  # (can be changed to Dice Loss if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop iterating over the DataLoader
for images, masks, original_sizes in train_dataloader:
    optimizer.zero_grad()

    # 1️⃣ Forward pass: Get predicted logits from the UNet model (batch_size, 1, 256, 256)
    logits = model(images)

    # 2️⃣ Resize the logits to match their original sizes before applying argmax
    resized_masks = resize_logits_before_argmax(logits, original_sizes)  

    # 3️⃣ Compute the loss for each mask individually
    total_loss = 0
    for pred_mask, gt_mask in zip(resized_masks, masks):
        loss = loss_fn(pred_mask, gt_mask)  # Compute loss for each individual sample
        total_loss += loss  # Accumulate loss for the batch
    
    # 4️⃣ Compute the average loss over the batch
    total_loss /= len(resized_masks)  

    # 5️⃣ Backpropagation and optimizer update
    total_loss.backward()
    optimizer.step()

    print(f"Loss: {total_loss.item()}")